# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [7]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 2.1365325017050654
 2.250450330954726
 1.948571473054109

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [8]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.273022  1.0       0.0
 0.604483  0.450926  1.0
U factor:
3×3 Matrix{Float64}:
 0.866289  0.858916   0.525245
 0.0       0.702491   0.819619
 0.0       0.0       -0.0981467

In [10]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [11]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [9]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.273022  1.0       0.0
 0.604483  0.450926  1.0

In [12]:
Alu.U

3×3 Matrix{Float64}:
 0.866289  0.858916   0.525245
 0.0       0.702491   0.819619
 0.0       0.0       -0.0981467

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [13]:
A \ b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999996
 1.0000000000000002

In [14]:
Alu \ b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999996
 1.0000000000000002

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [15]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [16]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.227523   0.898077  -0.376419
 -0.83335   -0.379549  -0.401833
 -0.503746   0.222263   0.834769
R factor:
3×3 Matrix{Float64}:
 -1.03953  -1.35008   -0.953501
  0.0       0.701298   0.796412
  0.0       0.0       -0.0819298

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [17]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.227523   0.898077  -0.376419
 -0.83335   -0.379549  -0.401833
 -0.503746   0.222263   0.834769

In [18]:
Aqr.R

3×3 Matrix{Float64}:
 -1.03953  -1.35008   -0.953501
  0.0       0.701298   0.796412
  0.0       0.0       -0.0819298

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [20]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.964376547675216
  0.06014076541452018
  4.272984442255042
vectors:
3×3 Matrix{Float64}:
  0.851786   0.00294211  -0.523881
 -0.402454  -0.636519    -0.657932
 -0.335396   0.771255    -0.540994

The values and the vectors can be extracted from the Eigen type by special indexing

In [21]:
AsymEig.values

3-element Vector{Float64}:
 -0.964376547675216
  0.06014076541452018
  4.272984442255042

In [22]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.851786   0.00294211  -0.523881
 -0.402454  -0.636519    -0.657932
 -0.335396   0.771255    -0.540994

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [23]:
inv(AsymEig) * Asym

3×3 Matrix{Float64}:
  1.0          6.66134e-16   1.22125e-15
 -1.06581e-14  1.0          -1.59872e-14
  1.24345e-14  1.24345e-14   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [24]:
n = 1000
A = randn(n, n)

1000×1000 Matrix{Float64}:
  0.65125     0.575828   0.777954  …   0.819985  -0.741345    1.25974
  0.424993    0.934442   1.19606      -0.559288  -1.28565    -0.210556
 -0.706565    1.04787    0.450017      0.169197   0.494301    0.875976
  0.0217484  -2.5516     0.542079     -0.188024   0.0105255   0.395571
  0.737864    1.08555    0.136698      1.44366    0.401552   -0.187047
 -0.800087    0.392526   0.480153  …   0.910282   1.50587    -0.915394
 -0.688406   -0.251067  -1.9127        1.32153   -0.676277    0.333143
 -1.61755    -0.459714   1.09717      -1.68276    0.0169059   2.56093
  0.318888   -1.18541   -1.13394      -1.49354    1.17518     1.90737
 -0.086665   -1.03605   -0.372626      0.270978  -0.523661    1.35147
  ⋮                                ⋱                         
 -2.43518    -1.11867   -0.386334     -0.485933   0.541026    0.973432
 -0.954265   -0.994552   0.889115     -1.17107   -1.46098    -1.12707
  0.0289814   0.306682  -0.547453     -0.979607  -0.216114   -0.

Julia can often infer special matrix structure

In [26]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [27]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

1.000820531243354

In [28]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [29]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [30]:
@time eigvals(Asym);

  0.440872 seconds (161.00 k allocations: 16.158 MiB, 57.59% compilation time)


In [34]:
@time eigvals(Asym_noisy)

  3.187321 seconds (13 allocations: 7.920 MiB)


1000-element Vector{Float64}:
 -87.8415457168614
 -87.63520940516858
 -86.63305519861616
 -86.3090559143807
 -86.06345614360599
 -85.51541607415338
 -85.28278660131697
 -84.60947206176876
 -84.25673372170017
 -83.72908281855607
   ⋮
  84.61012851616235
  85.07154907343232
  85.4299443447542
  85.81469474430547
  86.50158328939871
  86.86056310134936
  87.2621930948213
  87.75465724309313
  90.07527991085513

In [32]:
@time eigvals(Asym_explicit)

  0.548162 seconds (5.89 k allocations: 8.316 MiB, 2.27% compilation time)


1000-element Vector{Float64}:
 -87.84154571686119
 -87.63520940516929
 -86.63305519861646
 -86.3090559143815
 -86.0634561436053
 -85.5154160741528
 -85.28278660131768
 -84.60947206176853
 -84.25673372170137
 -83.72908281855577
   ⋮
  84.61012851616225
  85.07154907343248
  85.42994434475402
  85.81469474430598
  86.5015832893982
  86.86056310134961
  87.26219309482039
  87.75465724309146
  90.07527991085334

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [35]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.860978 seconds (800.66 k allocations: 224.330 MiB, 5.68% gc time, 27.49% compilation time)


6.331272031103927

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [37]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [38]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 4//5  4//5   1//5
 1//2  3//10  3//10
 1//5  4//5   9//10

In [39]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  9//5
 11//10
 19//10

In [40]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [41]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1  0//1
 1//4   1//1  0//1
 5//8  -1//3  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 4//5  4//5   1//5
 0//1  3//5  17//20
 0//1  0//1  11//24